# Homework 4
### Diversity Maximization on a Cloud

In [1]:
# Import Packages
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors
import numpy as np

In [2]:
conf = SparkConf().setAppName('HW4').setMaster('local')
sc = SparkContext(conf=conf)

<hr>
### <i> farthest first traversal</i> Function

In [14]:
import random
def farthest_first_traversal(P, k):
    idx_rnd = random.randint(0, len(P)-1)
    S = [P[idx_rnd]]
    related_center_idx = [idx_rnd for i in range(len(P))]
    dist_near_center = [Vectors.squared_distance(P[i], S[0]) for i in range(len(P))]

    for i in range(k-1):    
        new_center_idx = max(enumerate(dist_near_center), key=lambda x: x[1])[0] # argmax operation

        S.append(P[new_center_idx])

        for j in range(len(P)):
            if j != new_center_idx:
                dist = Vectors.squared_distance(P[j], S[-1])
                if dist < dist_near_center[j]:
                    dist_near_center[j] = dist
                    related_center_idx[j] = new_center_idx
            else:
                dist_near_center[j] = 0
                related_center_idx[j] = new_center_idx
    return S

<hr>
### Provided 2-Approximate Sequential Algorithm for Diversity Maximization

In [15]:
# points is a list of Vectors , k an integer
def runSequential(points, k):

    n = len(points)
    if k >= n:
        return points

    result = list()
    candidates = np.full(n , True)

    for iter in range(int(k / 2)):
        maxDist = 0.0
        maxI = 0
        maxJ = 0
        for i in range(n):
            if candidates[i] == True:
                for j in range(n):
                    d = Vectors.squared_distance(points[i], points[j])
                    if d > maxDist:
                        maxDist = d
                        maxI = i
                        maxJ = j
        result.append( points[maxI] )
        result.append( points[maxJ] )
        #print "selecting "+str(maxI)+" and "+str(maxJ)
        candidates[maxI] = False
        candidates[maxJ] = False

    if k % 2 != 0:
        s = np.random.randint(n)
        for i in range(n):
            j = (i + s) % n
            if candidates[j] == True:
                #print "selecting "+str(j)
                result.append( points[i] )
                break

    return result

In [16]:
input_length = 100
k = 11
d = 3
m = 10

test = []
for i in range(input_length):
    test.append( Vectors.dense([ np.random.randint(m) for i in range(d) ]) )

#print test
print("Input of size "+str(len(test))+" with k = " + str(k))
print("Selected: "+str(len(runSequential(test, k))))

Input of size 100 with k = 11
Selected: 11


<hr>
### <i>runMapReduce</i> Function
I have implemented a verstion without the MapReduce reasoning, just to compute the output

In [45]:
def runMapReduce(pointsrdd, k, numBlocks):
    
    # Partitioning
    block_size = len(pointsrdd) // numBlocks # floor
    blocks = [pointsrdd[i*block_size : (i+1)*block_size] for i in range(numBlocks)]
    
    # Extract k-Points using Farthest-First Traversal algorithm
    centers = [farthest_first_traversal(block, k) for block in blocks]
    
    # Gathering the Vectors
    coreset = [y for c in centers for y in c]
    
    return runSequential(coreset, k)

<hr>
### <i>measure</i> Function

In [7]:
def measure(pointslist):
        
    ### WRITE HERE THE CODE
    
    return